# More accurately concept blinding and removal of punctuation, normalizing digit, stop word removal

For the Semeval 2010 dataset, this means replacing entity words by entity1 and entity2. For DDI dataset, this means replacing the two involved drug words and the other drug words. For the i2b2 dataset, this means replacing the words by their concept names. This blinding is entirely dataset dependent and by having the replacement information present in the metadata of the original csv file generated by the converters, the pre-processing module can be entirely separated. 

The data pre-processing code has multiple assumptions. First of all, metadata that is generated should not contain any overlaps between words within the same entity itself (for example if e1 word_index is stored as [(3,3),(3,4)] that is not allowed, but [(3,3),(4,4)] is allowed. Further, if there is overlap between e1 and e2, they have to have an exact overlap and not a partial overlap. For example e1's word_index [(1,1),(2,2)] and e2's word_index [(1,1), (3,3)] is allowed but [(1,1), (2,2)] and [(1,2)] is not allowed. 

In [2]:
%load_ext autoreload

In [310]:
%autoreload

import os, pandas as pd, numpy as np
import sys
sys.path.append('../../../')
from relation_extraction.data import utils
import nltk
from ast import literal_eval
import itertools
RESOURCE_PATH = "/data/medg/misc/geeticka/relation_extraction/semeval2010"
indir = 'pre-processed/original/'
outdir1 = 'pre-processed/entity_blinding/'
outdir2 = 'pre-processed/punct_stop_digit/'
outdir3 = 'pre-processed/punct_digit/'
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.converters.converter_semeval2010 import write_dataframe, read_dataframe,\
check_equality_of_written_and_read_df, write_into_txt
# from relation_extraction.data.preprocess import replace_with_concept, replace_digit_punctuation_stop_word,\
# get_entity_positions_and_replacement_sentence
from relation_extraction.data.preprocess import preprocess

def makedir(outdir, res):
    if not os.path.exists(res(outdir)):
        os.makedirs(res(outdir))

A good row to look at in drugbank data is 4123 df_train_drugbank.iloc[4123]

## Write the different preprocessed versions into csv files

In [8]:
# original_dataframe_names = ['train', 'test']
# makedir(outdir1, res)
# makedir(outdir2, res)
# makedir(outdir3, res)

In [9]:
# for original_df_name in original_dataframe_names:
#     type1 = preprocess(read_dataframe, res(indir + original_df_name + '_original.csv'))
#     type2 = preprocess(read_dataframe, res(indir + original_df_name + '_original.csv'), 2)
#     type3 = preprocess(read_dataframe, res(indir + original_df_name + '_original.csv'), 3)
#     write_dataframe(type1, res(outdir1 + original_df_name + '_entity_blinding.csv'))
#     write_dataframe(type2, res(outdir2 + original_df_name + '_punct_stop_digit.csv'))
#     write_dataframe(type3, res(outdir3 + original_df_name + '_punct_digit.csv'))

## Write into text format

In [10]:
# for original_df_name in original_dataframe_names:
#     type1 = read_dataframe(res(outdir1 + original_df_name + '_entity_blinding.csv'))
#     type2 = read_dataframe(res(outdir2 + original_df_name + '_punct_stop_digit.csv'))
#     type3 = read_dataframe(res(outdir3 + original_df_name + '_punct_digit.csv'))
#     write_into_txt(type1, res(outdir1 + original_df_name + '_entity_blinding.txt'))
#     write_into_txt(type2, res(outdir2 + original_df_name + '_punct_stop_digit.txt'))
#     write_into_txt(type3, res(outdir3 + original_df_name + '_punct_digit.txt'))

Unique relations: 	 ['Component-Whole(e2,e1)' 'Other' 'Instrument-Agency(e2,e1)'
 'Member-Collection(e1,e2)' 'Cause-Effect(e2,e1)'
 'Entity-Destination(e1,e2)' 'Content-Container(e1,e2)'
 'Message-Topic(e1,e2)' 'Product-Producer(e2,e1)'
 'Member-Collection(e2,e1)' 'Entity-Origin(e1,e2)' 'Cause-Effect(e1,e2)'
 'Component-Whole(e1,e2)' 'Message-Topic(e2,e1)' 'Product-Producer(e1,e2)'
 'Entity-Origin(e2,e1)' 'Content-Container(e2,e1)'
 'Instrument-Agency(e1,e2)' 'Entity-Destination(e2,e1)']
Unique relations: 	 ['Component-Whole(e2,e1)' 'Other' 'Instrument-Agency(e2,e1)'
 'Member-Collection(e1,e2)' 'Cause-Effect(e2,e1)'
 'Entity-Destination(e1,e2)' 'Content-Container(e1,e2)'
 'Message-Topic(e1,e2)' 'Product-Producer(e2,e1)'
 'Member-Collection(e2,e1)' 'Entity-Origin(e1,e2)' 'Cause-Effect(e1,e2)'
 'Component-Whole(e1,e2)' 'Message-Topic(e2,e1)' 'Product-Producer(e1,e2)'
 'Entity-Origin(e2,e1)' 'Content-Container(e2,e1)'
 'Instrument-Agency(e1,e2)' 'Entity-Destination(e2,e1)']
Unique relatio

## Check that the lengths of the files created is correct

In [11]:
# def output_file_length(res, filename):
#     return len(open(res(filename)).readlines())

In [12]:
# print(output_file_length(res, indir + 'train_original.txt'))
# print(output_file_length(res, outdir1 + 'train_entity_blinding.txt'))
# print(output_file_length(res, outdir2 + 'train_punct_stop_digit.txt'))
# print(output_file_length(res, outdir3 + 'train_punct_digit.txt'))

8000
8000
8000
8000


In [13]:
# print(output_file_length(res, indir + 'test_original.txt'))
# print(output_file_length(res, outdir1 + 'test_entity_blinding.txt'))
# print(output_file_length(res, outdir2 + 'test_punct_stop_digit.txt'))
# print(output_file_length(res, outdir3 + 'test_punct_digit.txt'))

2717
2717
2717
2717


In [370]:
%autoreload
from relation_extraction.data.converters.converter_semeval2010 import read_dataframe
from relation_extraction.data.preprocess import get_new_sentence_with_entity_replacement, parse_position, \
entity1, entity2, entity_either, get_common_and_separate_entities, list_to_string, remove_whitespace, replace_ner as r_n
from spacy.tokens import Doc
import spacy
nlp = spacy.load('en')

In [312]:
df = read_dataframe(res(indir + 'train' + '_original.csv'))

In [313]:
df.iloc[0]

original_sentence     The system as described above has its greatest...
e1                                                        configuration
e2                                                             elements
relation_type                                    Component-Whole(e2,e1)
metadata              {'e1': {'word': 'configuration', 'word_index':...
tokenized_sentence    The system as described above has its greatest...
Name: 0, dtype: object

In [314]:
a = {1: '2'}

In [315]:
list(a.keys())

[1]

In [316]:
list(range(1,2))

[1]

In [317]:
not set([1]).intersection(set([2]))

True

## Ok good, seems like there is no overlap across named entities in the same sentence

In [318]:
# a method to check for overlap between the ner_dict that is created
def check_for_overlap(ner_dict):
    def expand_key(string): # a string that looks like '2:2' to [2]
        start = int(string.split(':')[0])
        end = int(string.split(':')[1])
        return list(range(start, end+1))
    expanded_keys = [expand_key(key) for key in ner_dict.keys()]
    for i1, item in enumerate(expanded_keys):
        for i2 in range(i1 + 1, len(expanded_keys)):
            if set(item).intersection(expanded_keys[i2]):
                return True # overlap is true
        for i2 in range(0, i1):
            if set(item).intersection(expanded_keys[i2]):
                return True
    return False
check_for_overlap({'3:4': 3, '2:6': 4})

True

In [319]:
parse_position('1:1')

(1, 1)

In [320]:
###
### Helper functions for the NER replacement
###
def overlap_index(index1, index2):
    def expand(index):
        start = index[0]
        end = index[1]
        return list(range(start, end+1))
    expand_index1 = expand(index1)
    expand_index2 = expand(index2)
    if set(expand_index1).intersection(set(expand_index2)):
        return True
    else: return False
    
# for indexes that look like (1,1) and (2,2) check if the left is fully included in the right
def fully_included(index1, index2):
    if index1[0] >= index2[0] and index1[1] <= index2[1]: return True
    else: return False

def beginning_overlap(index1, index2): # this is tricky when (1,1) and (2,2) are there
    if index1[0] < index2[0] and index1[1] <= index2[1]: return True
    else: return False

def end_overlap(index1, index2): # this is tricky
    if index1[0] >= index2[0] and index1[1] > index2[1]: return True
    else: return False
    
def beginning_and_end_overlap(index1, index2):
    if index1[0] < index2[0] and index1[1] > index2[1]: return True
    else:
        return False
#else there is no overlap
    
def correct_entity_indexes_with_ner(ner_dict, e_index):
    new_e_index = []
    for i in range(len(e_index)): # we are reading tuples here
        for key in ner_dict.keys():
            indexes = e_index[i]
            index2 = indexes
            index1 = parse_position(key) # checking if ner is fully included etc
            if not overlap(index1, index2): # don't do below if there is no overlap
                continue
            if beginning_overlap(index1, index2):
                e_index[i] = (index1[0], e_index[i][1])
            elif end_overlap(index1, index2):
                e_index[i] = (e_index[i][0], index1[1])
            elif beginning_and_end_overlap(index1, index2):
                e_index[i] = (index1[0], index1[1]) # else you don't change or do anything
    return e_index
            

The Facebook group was very upset by this behavior .

In [321]:
# given all of these dictionaries, return the ner replacement dictionary
def get_ner_replacement_dictionary(only_e1_index, only_e2_index, common_indexes, ner_dict):
    def update_dict_with_entity(e_index, ner_repl_dict, entity_name):
        for indexes in e_index:
            key1 = str(indexes[0]) + ':' + str(indexes[0])
            ner_repl_dict[key1] = {'replace_by': None, 'insert': entity_name + 'START'}
            key2 = str(indexes[-1] + 1) + ':' + str(indexes[-1] + 1)
            ner_repl_dict[key2] = {'replace_by': None, 'insert': entity_name + 'END'}
        return ner_repl_dict
    # we are going to do something different: only spans for NER will be counted, but
    # for the ENTITYSTART and ENTITYEND, we will keep the span as what token to insert before
    ner_repl_dict = {}
    for key in ner_dict:
        ner_repl_dict[key] = {'replace_by': ner_dict[key], 'insert': None}
    ner_repl_dict = update_dict_with_entity(only_e1_index, ner_repl_dict, entity1)
    ner_repl_dict = update_dict_with_entity(only_e2_index, ner_repl_dict, entity2)
    ner_repl_dict = update_dict_with_entity(common_indexes, ner_repl_dict, entity_either)
    return ner_repl_dict

In [322]:
# this function is different from the sort_position_keys because
# we care about sorting not just by the beginning token, but also by the length that the span contains
def ner_sort_position_keys(ner_repl_dict): # this can potentially replace sort_position_keys
    # but only if the application of this function does not change the preprocessed CSVs generated
    def len_key(key):
        pos = parse_position(key)
        return pos[1] - pos[0] + 1
    positions = list(ner_repl_dict.keys())
    sorted_positions = sorted(positions, key=lambda x: (parse_position(x)[0], len_key(x)))
    return sorted_positions

In [323]:
ner_sort_position_keys({'1:3':3, '1:1':5, '1:2':4, '4:5':4})

['1:1', '1:2', '1:3', '4:5']

In [343]:
def replace_ner(row, check_ner_overlap=False): # similar to concept_replace, with some caveats
    named_entities = 0
    sentence = row.tokenized_sentence.split()
    e1_indexes = row.metadata['e1']['word_index']
    e2_indexes = row.metadata['e2']['word_index']
    only_e1_indexes, only_e2_indexes, common_indexes = \
    get_common_and_separate_entities(e1_indexes, e2_indexes)
#     sentence = get_new_sentence_with_entity_replacement(sentence, e1_indexes, e2_indexes)
    tokenizedSentence = sentence # in this case lowercasing is not helpful
    doc = Doc(nlp.vocab, words=tokenizedSentence)
    nlp.tagger(doc)
    nlp.parser(doc)
    nlp.entity(doc) # run NER
    ner_dict = {} # first test for overlaps within ner
    for ent in doc.ents:
        named_entities += 1
        key = str(ent.start) + ':' + str(ent.end - 1)
        ner_dict[key] = ent.label_
    if check_ner_overlap and check_for_overlap(ner_dict):
        print("There is overlap", ner_dict) # only need to check this once
    #Below code works only if there isn't overlap within ner_dict, so make sure that there isn't overlap
    
    # overlaps between ner label and e1 and e2 indexes are a problem. 
        # Type 1: NER overlaps with e1 or e2 in the beginning or end
        # Here we want to keep the NER link the same but extend e1 or e2 index to the beginning or end of the
        # NER 
        
    print(only_e1_indexes, only_e2_indexes, common_indexes, ner_dict)
    only_e1_indexes = correct_entity_indexes_with_ner(ner_dict, only_e1_indexes)
    only_e2_indexes = correct_entity_indexes_with_ner(ner_dict, only_e2_indexes)
    common_indexes = correct_entity_indexes_with_ner(ner_dict, common_indexes)
        
        #Type 2: NER is inside of the entity completely: At this point it should be simply ok to mention at what 
        # token to insert ENTITYstart and ENTITYend
    ner_repl_dict = get_ner_replacement_dictionary(only_e1_indexes, only_e2_indexes, common_indexes,
                                                  ner_dict)
    print(ner_repl_dict)
    print(only_e1_indexes, only_e2_indexes, common_indexes)
    sorted_positions = ner_sort_position_keys(ner_repl_dict)
    print('sorted_positions', sorted_positions)
    print(sentence)
    new_sentence = '' # this below part is buggy, shouldn't be too bad to fix 
    for i in range(len(sorted_positions)):
        curr_pos = sorted_positions[i]
        curr_start_pos, curr_end_pos = parse_position(curr_pos)
        curr_dict = ner_repl_dict[curr_pos]
        start_insert = '' if curr_dict['insert'] is None else curr_dict['insert'].upper()
        between_replace = list_to_string(sentence[curr_start_pos: curr_end_pos + 1]) \
        if curr_dict['replace_by'] is None else curr_dict['replace_by']
        if i == 0:
            new_sentence += list_to_string(sentence[:curr_start_pos]) + ' ' + start_insert + ' ' + \
            between_replace + ' '
        else:
            prev_pos = sorted_positions[i-1]
            _, prev_end_pos = parse_position(prev_pos)
            middle = list_to_string(sentence[prev_end_pos+1 : curr_start_pos]) # refers to middle between prev
            # segment and the current segment
            if middle == '':
                middle = ' '
            new_sentence += middle + ' ' + start_insert + ' ' + between_replace + ' '
            if i == len(sorted_positions) - 1 and curr_end_pos < len(sentence) - 1:
                new_sentence += ' ' + list_to_string(sentence[curr_end_pos+1:])
    new_sentence = remove_whitespace(new_sentence)
    return new_sentence
    # dictionary that we generate for sentence representation should look like:
    # {'0:0': {'replace_by': NER, 'insert': NONE}, '1:1': {'replace_by': None, 'insert': 'ENTITYSTART'}}
    # In this case, sorted positions should be arranged by first number but also keep in mind how many 
    # words are included in these. 

    
    
    

In [373]:
r_n(df.iloc[900])

"In her dentist dad 's office , with the help of her brother PERSON she finds a ESTART bottle EEND full of pulled EOTHERSTART teeth EOTHEREND ."

In [374]:
replace_ner(df.iloc[900])

[(17, 17)] [(21, 21)] [] {'13:13': 'PERSON'}
{'13:13': {'replace_by': 'PERSON', 'insert': None}, '17:17': {'replace_by': None, 'insert': 'ESTART'}, '18:18': {'replace_by': None, 'insert': 'EEND'}, '21:21': {'replace_by': None, 'insert': 'EOTHERSTART'}, '22:22': {'replace_by': None, 'insert': 'EOTHEREND'}}
[(17, 17)] [(21, 21)] []
sorted_positions ['13:13', '17:17', '18:18', '21:21', '22:22']
['In', 'her', 'dentist', 'dad', "'s", 'office', ',', 'with', 'the', 'help', 'of', 'her', 'brother', 'Norman', 'she', 'finds', 'a', 'bottle', 'full', 'of', 'pulled', 'teeth', '.']


"In her dentist dad 's office , with the help of her brother PERSON she finds a ESTART bottle EEND full of pulled EOTHERSTART teeth EOTHEREND ."

In [308]:
df.iloc[6]['tokenized_sentence']

'The current view is that the chronic inflammation in the distal part of the stomach caused by Helicobacter pylori infection results in an increased acid production from the non - infected upper corpus region of the stomach .'

In [122]:
named_entities = 0

In [377]:
named_entities = 0
df['tagged_sentence'] = df.apply(r_n, args=(False,), axis=1)

In [378]:
df['tagged_sentence']

0       The system as described above has its greatest...
1       The ESTART child EEND was carefully wrapped an...
2       The ESTART author EEND of a keygen uses a EOTH...
3       A misty ESTART ridge EEND uprises from the EOT...
4       The ESTART student EOTHERSTART association EOT...
5       This is the sprawling ESTART complex EEND that...
6       The current view is that the chronic ESTART in...
7       ESTART People EEND have been moving back into ...
8       The ESTART lawsonite EEND was contained in a E...
9       The solute was placed inside a beaker and CARD...
10      The CARDINAL ESTART essays EEND collected in t...
11      Their ESTART composer EEND has sunk into EOTHE...
12      The ORG issues an official ESTART citation EEN...
13      The ESTART burst EEND has been caused by water...
14      Even commercial ESTART networks EEND have move...
15      It was a friendly ESTART call EEND to remind t...
16      GPE - born ESTART virtuoso EEND finds harmony ...
17      The ES

In [124]:
named_entities

0

In [125]:
df = read_dataframe(res(indir + 'test' + '_original.csv'))

In [126]:
df.apply(replace_ner, axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2687    None
2688    None
2689    None
2690    None
2691    None
2692    None
2693    None
2694    None
2695    None
2696    None
2697    None
2698    None
2699    None
2700    None
2701    None
2702    None
2703    None
2704    None
2705    None
2706    None
2707    None
2708    None
2709    None
2710    None
2711    None
2712    None
2713    None
2714    None
2715    None
2716    None
Length: 2717, dtype: object